In [ ]:
################################  Storage_Optimization.ipynb  ####################################
# Author: Phind
# Description: TEST Code
# Data: 24-Nov-2024
#################################################################################

In [ ]:
import pandas as pd
import numpy as np


# Data Import and Clean

In [ ]:
#### Read FILE:: (AKINS FoMoCo_Piece_Sales_112222_YTD.xlsx) into Dataframe
df_Akins = utils.read_excel(AKINS_FOMO_FILE_PATH)
df_Akins
#utils.print_df(df_Wholesale_Ford) # Print the Dataframe

In [ ]:
#### Read FILE:: (GPARTS Part Measures.xlsx) into Dataframe
df_Gparts = utils.read_excel(GPARTS_FILE_PATH)
utils.print_df(df_Gparts) # Print the Dataframe

print(df_Gparts[df_Gparts["Prod Att - Length"] == 0].shape[0]) # Print the Rows with 0 Dimensions

In [ ]:
#### Read FILE:: (Wholesale JAN_Oct_Parts_Ranking_Counter_Invoices_All_Brands.xlsx) into Dataframe
df_Wholesale = utils.read_excel(WHOLESALE_FILE_PATH)

# Clean the Wholesale Dataframe
df_Wholesale = df_Wholesale.drop(columns=[col for col in df_Wholesale.columns if 'Unnamed' in col], inplace=False)
df_Wholesale_Ford = df_Wholesale[df_Wholesale['Vendor'] == 'FOR'] # Put only 'Ford' Brand Data into another DF

utils.print_df(df_Wholesale_Ford) # Print the Dataframe

In [ ]:
#### Read FILE:: (Service JAN_Oct_Parts_Ranking_ROs_All_Brands.xlsx) into Dataframe
df_Service = utils.read_excel(SERVICE_FILE_PATH)

# Clean the Service Dataframe
df_Service = df_Service.drop(columns=[col for col in df_Service.columns if 'Unnamed' in col], inplace=False)
df_Service_Ford = df_Service[df_Service['Vendor'] == 'FOR'] # Put only 'Ford' Brand Data into another DF

utils.print_df(df_Service_Ford, 100) # Print the Dataframe

In [ ]:
high_density_drawers = {
    '36":4': {'width': 9, 'depth': 24},
    '48":4': {'width': 12, 'depth': 24},
    '48":6': {'width': 8, 'depth': 24}
}

clip_shelving = {
    '12":0': {'height': 15, 'depth': 12, 'width': 36},
    '18":0': {'height': 15, 'depth': 18, 'width': 48},
    '24":0': {'height': 15, 'depth': 24, 'width': 48}
}

bulk_storage = {
    '24":0': {'height': 12, 'depth': 24, 'width': 48},
    '36":0': {'height': 12, 'depth': 36, 'width': 72},
    '42":0': {'height': 12, 'depth': 42, 'width': 72},
    '48":0': {'height': 12, 'depth': 48, 'width': 96},
    '72":0': {'height': 12, 'depth': 72, 'width': 96}
}


##### Step 3: Implement the storage assignment function
##### Now, let's create a function that assigns parts to storage based on their dimensions and sales frequency zone:

In [ ]:
def assign_storage(part_dimensions, part_height, part_width, part_depth, sales_frequency):
    zones = ['Red Hot', 'Orange', 'Yellow', 'Green', 'Blue']
    zone_index = int(sales_frequency * 4)  # Assuming 0-1 scale for sales frequency
    
    if part_height <= 6 and part_width <= 12 and part_depth <= 24:
        return 'High-Density Drawer'
    
    if part_height <= 15:
        for size, specs in clip_shelving.items():
            if part_width <= specs['width'] and part_depth <= specs['depth']:
                return f'Clip Shelving {size}'
    
    if part_height >= 12:
        for size, specs in bulk_storage.items():
            if part_width <= specs['width'] and part_depth <= specs['depth']:
                return f'Bulk Storage {size}'
    
    # Specialty storage (Battery, Tire, Hanging)
    if 'battery' in part_name.lower():
        return 'Battery Storage'
    elif 'tire' in part_name.lower():
        tire_diameter = float(part_name.split(' ')[0])  # Assuming diameter is first number
        if tire_diameter <= 28:
            return 'Tire Carousel (28" or less)'
        elif 28 < tire_diameter <= 33:
            return 'Tire Carousel (28-33")'
        else:
            return 'Tire Carousel (>33")'
    elif part_width > 24 and part_height < 4:
        return 'Hanging Storage'
    
    # Bumper Cover Storage
    if 'bumper cover' in part_name.lower():
        return 'Bumper Cover Storage'
    
    # Default assignment
    return 'Other Storage'


# Data Analysis


In [ ]:
## Find Rows with 0 in either Dimensions

# Print the Rows with 0 Dimensions
print(f"Numer of Rows with 0 Dimensions: {df_Gparts[df_Gparts["Prod Att - Length"] == 0].shape[0]}, {(df_Gparts[df_Gparts["Prod Att - Length"] == 0].shape[0]/df_Gparts.shape[0])*100}%") 
utils.print_df(df_Gparts[df_Gparts["Prod Att - Length"] == 0], 10) # Print top 10 Rows with 0 Dimensions

# Data Processing & Calculation

In [ ]:
## Sort the 'Wholesale..' DF by 'Sort Pcs' Columns in Descending order
# Sold Pcs/Sold = Sales Frequency

df_Wholesale_Ford["Total Sold"] = df_Wholesale_Ford["Sold"] +  df_Wholesale_Ford["Sold.1"]
df_Wholesale_Ford.sort_values(['Total Sold'], ascending=False, inplace=True)

In [ ]:
df_Wholesale_Ford

In [ ]:
data = []
sum = 0
totalSoldPCs = int(df_Wholesale_Ford["Total Sold"].sum())
print(sum, totalSoldPCs)
for i in range(df_Wholesale_Ford.shape[0]):
    zone = ""
    if sum/totalSoldPCs <= 0.2:
        zone = "Red Hot Zone"
    if sum/totalSoldPCs > 0.2 and sum/totalSoldPCs <= 0.4:
        zone = "Orange Zone"
    if sum/totalSoldPCs > 0.4 and sum/totalSoldPCs <= 0.6:
        zone = "Yellow Zone"
    if sum/totalSoldPCs > 0.6 and sum/totalSoldPCs <= 0.8:
        zone = "Green Zone"
    if sum/totalSoldPCs > 0.8:
        zone = "Blue Zone"
    data.append([df_Wholesale_Ford["Part Number"].iloc[i], zone, sum])
    sum = sum + df_Wholesale_Ford["Total Sold"].iloc[i]
df_zones = pd.DataFrame(data)
utils.print_df(df_zones,None)

with open("htt.txt", "w") as f:
    f.write(df_zones.to_string())